### 🤖 ¿Qué son los Sistemas RAG Multi-Agente?
Un Sistema RAG Multi-Agente divide el pipeline RAG en múltiples agentes especializados — cada uno responsable de un rol específico — y les permite colaborar en una sola consulta o tarea.

#### 1. 📋 Sistema de Red Multi-Agente RAG con LangGraph
Descripción del Proyecto

Un sistema de Generación Aumentada por Recuperación (RAG) amigable para principiantes que utiliza una arquitectura multi-agente para responder preguntas inteligentemente desde tus documentos. Construido con LangGraph v0.3 para orquestación de flujos de trabajo y OpenAI para comprensión del lenguaje.

Qué Hace

Transforma tus documentos (PDFs, archivos de texto) en una base de conocimiento buscable que puede responder preguntas inteligentemente usando IA. Simplemente carga documentos y haz preguntas en lenguaje natural - el sistema encuentra información relevante y genera respuestas completas.

Características Clave

- 📚 Soporte Multi-Formato: Maneja documentos PDF y de texto
- 🤖 Arquitectura de 3 Agentes: Agentes especializados para procesamiento de documentos, recuperación y generación de respuestas
- 🔍 Búsqueda Inteligente: Búsqueda semántica basada en vectores encuentra información relevante
- 💬 Preguntas y Respuestas en Lenguaje Natural: Haz preguntas en español simple

In [ ]:
# Importación del módulo os para interactuar con variables de entorno del sistema
import os

# Importación de load_dotenv para cargar variables de entorno desde archivo .env
from dotenv import load_dotenv

# Carga las variables de entorno desde el archivo .env en el directorio actual
# Esto permite mantener las API keys seguras fuera del código
load_dotenv()

# Importación de init_chat_model para inicializar modelos de chat de diferentes proveedores
from langchain.chat_models import init_chat_model

# Configuración de la API key de Tavily (motor de búsqueda web para IA)
# Tavily proporciona resultados de búsqueda optimizados para LLMs
os.environ["TAVILY_API_KEY"]=os.getenv("TAVILY_API_KEY")

# Configuración de la API key de OpenAI para acceder a modelos GPT
# Esta key permite autenticarse con los servicios de OpenAI
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")

In [ ]:
# Inicialización del modelo de lenguaje usando GPT-4o-mini de OpenAI
# GPT-4o-mini es una versión más económica y rápida de GPT-4
# Ideal para sistemas multi-agente donde se hacen múltiples llamadas al LLM
llm=init_chat_model("openai:gpt-4o-mini")

# Muestra el objeto LLM configurado con sus parámetros
llm

In [ ]:
# Importación de Annotated para agregar metadatos a tipos en type hints
from typing import Annotated

# Importación de TavilySearch para búsquedas web optimizadas para IA
from langchain_tavily import TavilySearch

# Importación de tool decorator para convertir funciones en herramientas de LangChain
from langchain_core.tools import tool

# Importación de WikipediaQueryRun para ejecutar búsquedas en Wikipedia
from langchain.tools import WikipediaQueryRun

# Importación de WikipediaAPIWrapper, wrapper que encapsula la API de Wikipedia
from langchain.utilities import WikipediaAPIWrapper

# Importación de TextLoader para cargar archivos de texto plano
from langchain.document_loaders import TextLoader

# Importación de FAISS para búsqueda vectorial eficiente
from langchain.vectorstores import FAISS

# Importación de OpenAIEmbeddings para generar vectores de embeddings
from langchain_openai import OpenAIEmbeddings

# Importación de RecursiveCharacterTextSplitter para dividir textos en chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
# Creación de una instancia de TavilySearch configurada para búsqueda web
# max_results=5: limita los resultados a los 5 más relevantes
# Tavily es un motor de búsqueda diseñado específicamente para aplicaciones de IA
# Proporciona resultados limpios y optimizados para consumo por LLMs
tavily_tool=TavilySearch(max_results=5)

In [ ]:
### Función genérica para crear una herramienta de recuperación desde texto

# Importación de Tool para crear herramientas personalizadas de LangChain
from langchain.agents import Tool

def make_retriever_tool_from_text(file, name, desc):
    """
    Crea una herramienta de recuperación personalizada desde un archivo de texto.
    
    Parámetros:
    - file: ruta del archivo de texto a cargar
    - name: nombre de la herramienta
    - desc: descripción de lo que hace la herramienta
    
    Retorna: objeto Tool configurado con recuperación vectorial
    """
    
    # Carga del archivo de texto usando TextLoader con codificación UTF-8
    # Esto asegura la correcta lectura de caracteres especiales y acentos
    docs=TextLoader(file, encoding="utf-8").load()
    
    # División de los documentos en chunks de 500 caracteres
    # chunk_overlap=50: superposición de 50 caracteres entre chunks para mantener contexto
    chunks = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50).split_documents(docs)
    
    # Creación de un vector store FAISS con los chunks
    # FAISS indexa los embeddings para búsquedas rápidas de similitud
    vs = FAISS.from_documents(chunks, OpenAIEmbeddings())
    
    # Conversión del vector store en retriever para búsquedas
    retriever = vs.as_retriever()

    # Definición de la función interna que ejecutará la herramienta
    def tool_func(query: str) -> str:
        """
        Función que realiza la búsqueda vectorial.
        
        Parámetros:
        - query: consulta de búsqueda
        
        Retorna: contenido de los documentos encontrados concatenados
        """
        # Imprime mensaje indicando qué herramienta se está usando
        print(f"📚 Usando herramienta: {name}")
        
        # Invoca el retriever para buscar documentos relevantes
        results = retriever.invoke(query)
        
        # Concatena el contenido de todos los documentos encontrados
        # Separados por doble salto de línea para mejor legibilidad
        return "\n\n".join(doc.page_content for doc in results)
    
    # Retorna un objeto Tool de LangChain con la función configurada
    return Tool(name=name, description=desc, func=tool_func)


# Creación de una herramienta de recuperación para notas de investigación internas
# Esta herramienta busca en el archivo internal_docs.txt
internal_tool_1=make_retriever_tool_from_text(
    "internal_docs.txt",  # Archivo de documentos internos
    "InternalResearchNotes",  # Nombre de la herramienta
    "Buscar en notas de investigación internas para resultados experimentales"  # Descripción
)

# Muestra la herramienta creada
internal_tool_1

In [ ]:
# Importaciones necesarias para crear agentes multi-agente

# BaseMessage: clase base para todos los mensajes en LangChain
# HumanMessage: representa mensajes del usuario/humano
from langchain_core.messages import BaseMessage, HumanMessage

# create_react_agent: crea agentes con patrón ReAct (Reason + Act)
from langgraph.prebuilt import create_react_agent

# MessagesState: estado que contiene historial de mensajes
# END: marcador especial que indica el final del grafo
from langgraph.graph import MessagesState, END

# Command: clase para enviar comandos de actualización y navegación en el grafo
from langgraph.types import Command

def get_next_node(last_message: BaseMessage, goto: str):
    """
    Determina el siguiente nodo a ejecutar en el grafo multi-agente.
    
    Esta función implementa la lógica de terminación:
    - Si el mensaje contiene "FINAL ANSWER", termina el flujo
    - Si no, continúa al siguiente nodo especificado
    
    Parámetros:
    - last_message: último mensaje generado por un agente
    - goto: nombre del siguiente nodo a ejecutar
    
    Retorna: END si el trabajo está completo, o el nombre del siguiente nodo
    """
    # Verifica si el mensaje contiene "FINAL ANSWER" (en mayúsculas)
    # Este es el indicador de que algún agente ha completado el trabajo
    if "FINAL ANSWER" in last_message.content:
        # Cualquier agente decidió que el trabajo está completo
        return END
    
    # Si no hay "FINAL ANSWER", continúa al siguiente nodo
    return goto

In [ ]:
def make_system_prompt(suffix: str) -> str:
    """
    Crea un prompt del sistema para agentes colaborativos.
    
    Este prompt establece las reglas de colaboración entre agentes:
    - Cada agente es parte de un equipo
    - Pueden usar herramientas específicas
    - Deben progresar hacia la respuesta final
    - Pueden pasar el trabajo a otro agente
    - Deben indicar cuando tienen la respuesta final
    
    Parámetros:
    - suffix: instrucciones adicionales específicas para cada agente
    
    Retorna: string con el prompt completo del sistema
    """
    return (
        # Establece el rol del agente: asistente colaborativo
        "Eres un asistente de IA útil, colaborando con otros asistentes."
        
        # Instrucción para usar herramientas disponibles
        " Usa las herramientas proporcionadas para progresar hacia responder la pregunta."
        
        # Permite que el agente no complete toda la tarea solo
        " Si no puedes responder completamente, está bien, otro asistente con diferentes herramientas"
        " ayudará donde lo dejaste. Ejecuta lo que puedas para hacer progreso."
        
        # Instrucción crucial: cómo indicar que el trabajo está completo
        " Si tú o cualquiera de los otros asistentes tienen la respuesta final o el entregable,"
        " prefija tu respuesta con RESPUESTA FINAL para que el equipo sepa que debe detenerse."
        
        # Agrega instrucciones específicas del agente
        f"\n{suffix}"
    )

In [ ]:
### Agente de Investigación y nodo

# Creación del agente de investigación usando el patrón ReAct
# Este agente es especializado en hacer investigación usando herramientas específicas
research_agent=create_react_agent(
    llm,  # Modelo de lenguaje GPT-4o-mini configurado anteriormente
    
    # Lista de herramientas disponibles para este agente:
    tools=[
        internal_tool_1,  # Herramienta para buscar en documentos internos
        tavily_tool  # Herramienta para búsqueda web con Tavily
    ],
    
    # Prompt del sistema que define el rol y comportamiento del agente
    prompt=make_system_prompt(
        # Instrucciones específicas para el agente de investigación
        "Solo puedes hacer investigación. Usa la herramienta con la que estás vinculado, puedes usar ambas."
        " Estás trabajando con un colega escritor de contenido."
    )
)

# Muestra el agente de investigación configurado
research_agent

In [ ]:
## Nodo de Investigación

# Importación de Literal para type hints con valores literales específicos
from typing import Literal

def research_node(state: MessagesState) -> Command[Literal["blog_generator", END]]:
    """
    Nodo que ejecuta el agente de investigación y decide el siguiente paso.
    
    Flujo:
    1. Invoca el agente de investigación con el estado actual
    2. Determina si el trabajo está completo o debe pasar a otro agente
    3. Convierte el mensaje del agente a HumanMessage para compatibilidad
    4. Retorna comando con actualización de estado y siguiente nodo
    
    Parámetros:
    - state: estado actual con historial de mensajes
    
    Retorna: Command dirigiendo al generador de blogs o END
    """
    
    # Invoca el agente de investigación con el estado actual
    # El agente procesa los mensajes y usa sus herramientas (internal_tool_1, tavily_tool)
    result = research_agent.invoke(state)
    
    # Determina el siguiente nodo basándose en el último mensaje
    # Si contiene "FINAL ANSWER" → END, si no → "blog_generator"
    goto = get_next_node(result["messages"][-1], "blog_generator")

    # Envuelve el último mensaje en un HumanMessage
    # Esto es necesario porque no todos los proveedores de LLM permiten
    # mensajes de IA en la última posición de la lista de mensajes de entrada
    result["messages"][-1] = HumanMessage(
        content=result["messages"][-1].content,  # Contenido del mensaje del agente
        name="researcher"  # Identifica que este mensaje viene del investigador
    )
    
    # Retorna un Command que:
    # 1. Actualiza el estado con el historial de mensajes del agente de investigación
    # 2. Navega al siguiente nodo (blog_generator o END)
    return Command(
        update={
            # Comparte el historial de mensajes interno del agente de investigación
            # con otros agentes para mantener el contexto
            "messages": result["messages"],
        },
        goto=goto,  # Siguiente nodo a ejecutar
    )


In [ ]:
### Agente de Escritura de Blog

# Creación del agente de escritura de blogs usando el patrón ReAct
# Este agente se especializa en escribir contenido detallado basándose en investigación
blog_agent=create_react_agent(
    llm,  # Modelo de lenguaje GPT-4o-mini
    
    # Lista de herramientas: vacía porque este agente no usa herramientas externas
    # Su función es procesar información y escribir, no buscar
    tools=[],
    
    # Prompt del sistema que define el rol específico del escritor
    prompt=make_system_prompt(
        # Instrucciones específicas: solo escribir, no investigar
        "Solo puedes escribir un blog detallado. Estás trabajando con un colega investigador."
    )
)

def blog_node(state: MessagesState) -> Command[Literal["researcher", END]]:
    """
    Nodo que ejecuta el agente de escritura de blogs.
    
    Flujo:
    1. Invoca el agente de blog con el estado (que incluye la investigación)
    2. Determina si el blog está completo o necesita más investigación
    3. Convierte el mensaje a HumanMessage para compatibilidad
    4. Retorna comando con actualización y siguiente paso
    
    Parámetros:
    - state: estado actual con historial de mensajes (incluye investigación)
    
    Retorna: Command dirigiendo al investigador o END
    """
    
    # Invoca el agente de blog con el estado actual
    # El agente procesa la información de investigación y genera contenido escrito
    result = blog_agent.invoke(state)
    
    # Determina el siguiente nodo
    # Si el blog está completo (contiene "FINAL ANSWER") → END
    # Si necesita más información → vuelve al "researcher"
    goto = get_next_node(result["messages"][-1], "researcher")
    
    # Envuelve el último mensaje en un HumanMessage
    # Marca el mensaje como proveniente del "blog_generator"
    result["messages"][-1] = HumanMessage(
        content=result["messages"][-1].content,  # Contenido del blog generado
        name="blog_generator"  # Identifica el autor del mensaje
    )
    
    # Retorna Command con actualización de estado y navegación
    return Command(
        update={
            # Comparte el historial de mensajes interno del agente de blog
            # con otros agentes
            "messages": result["messages"],
        },
        goto=goto,  # Siguiente nodo (researcher o END)
    )

In [ ]:
# Importaciones para construcción del grafo
from langgraph.graph import StateGraph, START

# Creación del grafo de estado con MessagesState
# MessagesState mantiene el historial de mensajes entre agentes
workflow = StateGraph(MessagesState)

# Agregado de nodos al grafo
# Cada nodo representa un agente especializado

# Nodo del investigador: busca información usando herramientas
workflow.add_node("researcher", research_node)

# Nodo del generador de blogs: escribe contenido basándose en la investigación
workflow.add_node("blog_generator", blog_node)

# Define el punto de entrada del grafo
# El flujo comienza con el investigador
workflow.add_edge(START, "researcher")

# Compilación del grafo para hacerlo ejecutable
# .compile() valida la estructura y crea el grafo final
graph = workflow.compile()

# Muestra el grafo compilado (en Jupyter puede renderizar una visualización)
graph

In [ ]:
# Invocación del grafo multi-agente con una consulta
# Este es el punto de entrada para usar el sistema

response=graph.invoke({
    # El mensaje inicial que dispara todo el flujo de trabajo
    # "messages" es el campo requerido por MessagesState
    "messages": "Escribe un blog detallado sobre variantes de transformers en despliegues de producción"
})

# Flujo de ejecución esperado:
# 1. START → researcher (nodo de investigación)
# 2. researcher busca información usando internal_tool_1 y tavily_tool
# 3. researcher → blog_generator (pasa la investigación al escritor)
# 4. blog_generator escribe el blog usando la información recopilada
# 5. Si está completo: añade "FINAL ANSWER" y termina (END)
# 6. Si necesita más info: vuelve a researcher

# La salida mostrará: "📚 Usando herramienta: InternalResearchNotes"
# cuando el investigador use la herramienta de documentos internos

In [ ]:
# Extrae y muestra el contenido del último mensaje
# Este es el blog final generado por el sistema multi-agente
# [-1] accede al último elemento de la lista de mensajes
# .content obtiene el texto del mensaje
response["messages"][-1].content

### Supervisor Multi-Agente con RAG
El Supervisor es una arquitectura multi-agente donde agentes especializados son coordinados por un agente supervisor central. El agente supervisor controla todo el flujo de comunicación y delegación de tareas, tomando decisiones sobre qué agente invocar basándose en el contexto actual y los requisitos de la tarea.

En este tutorial, construirás un sistema supervisor con dos agentes — un experto en investigación y matemáticas. Al final del tutorial podrás:

1. Construir agentes especializados de investigación y matemáticas
2. Construir un supervisor para orquestarlos con langgraph-supervisor pre-construido
3. Construir un supervisor personalizado
4. Implementar delegación avanzada de tareas

In [ ]:
# Muestra la herramienta de recuperación interna creada anteriormente
# Esta herramienta será usada por el agente de investigación en el sistema supervisor
internal_tool_1

In [ ]:
# Importación de TavilySearch para búsquedas web
from langchain_tavily import TavilySearch

# Creación de herramienta de búsqueda web con límite de 3 resultados
# Limitamos a 3 para reducir el contexto y el costo de tokens
web_search = TavilySearch(max_results=3)

# Muestra la herramienta de búsqueda web configurada
web_search

In [ ]:
# Importación de create_react_agent para crear agentes con patrón ReAct
from langgraph.prebuilt import create_react_agent

# Creación del agente de investigación para el sistema supervisor
# Este agente tiene capacidades más restringidas que en el ejemplo anterior
research_agent=create_react_agent(
    model=llm,  # Modelo GPT-4o-mini
    
    # Herramientas disponibles: búsqueda web y documentos internos
    tools=[web_search, internal_tool_1],
    
    # Prompt más específico para este contexto
    prompt=(
        "Eres un agente de investigación.\n\n"
        "INSTRUCCIONES:\n"
        
        # Restricción clara: SOLO investigación, NO matemáticas
        "- Asiste SOLO con tareas relacionadas con investigación, NO hagas ninguna matemática\n"
        
        # Después de terminar, debe reportar directamente al supervisor
        "- Después de que termines con tus tareas, responde al supervisor directamente\n"
        
        # Solo debe proporcionar resultados, sin texto adicional
        "- Responde SOLO con los resultados de tu trabajo, NO incluyas NINGÚN otro texto."
    ),
    
    # Nombre del agente para identificación en mensajes
    name="research_agent"
)

# Muestra el agente de investigación configurado
research_agent

In [ ]:
# Definición de funciones matemáticas básicas como herramientas
# Estas funciones serán usadas por el agente de matemáticas

def add(a: float, b: float):
    """
    Suma dos números.
    
    Parámetros:
    - a: primer número (float)
    - b: segundo número (float)
    
    Retorna: la suma de a y b
    """
    return a + b


def multiply(a: float, b: float):
    """
    Multiplica dos números.
    
    Parámetros:
    - a: primer número (float)
    - b: segundo número (float)
    
    Retorna: el producto de a y b
    """
    return a * b


def divide(a: float, b: float):
    """
    Divide dos números.
    
    Parámetros:
    - a: numerador (float)
    - b: denominador (float)
    
    Retorna: el cociente de a dividido por b
    
    Nota: No maneja división por cero explícitamente
    """
    return a / b


# Creación del agente de matemáticas
# Este agente se especializa en operaciones matemáticas
math_agent=create_react_agent(
    model=llm,  # Modelo GPT-4o-mini
    
    # Herramientas matemáticas disponibles
    tools=[add, multiply, divide],
    
    # Prompt con instrucciones específicas para el agente matemático
    prompt=(
        "Eres un agente de matemáticas.\n\n"
        "INSTRUCCIONES:\n"
        
        # Restricción: SOLO matemáticas
        "- Asiste SOLO con tareas relacionadas con matemáticas\n"
        
        # Reportar al supervisor cuando termine
        "- Después de que termines con tus tareas, responde al supervisor directamente\n"
        
        # Solo resultados, sin texto adicional
        "- Responde SOLO con los resultados de tu trabajo, NO incluyas NINGÚN otro texto."
    ),
    
    # Nombre del agente
    name="math_agent"
)

In [ ]:
### Crear agente supervisor

# Importación de create_supervisor desde langgraph_supervisor
# Esta es una función pre-construida que facilita la creación de supervisores
from langgraph_supervisor import create_supervisor

# Creación del supervisor que coordina a los agentes
supervisor=create_supervisor(
    model=llm,  # Modelo LLM para el supervisor (GPT-4o-mini)
    
    # Lista de agentes que el supervisor puede delegar trabajo
    agents=[research_agent, math_agent],
    
    # Prompt que define el comportamiento del supervisor
    prompt=(
        "Eres un supervisor gestionando dos agentes:\n"
        
        # Describe cada agente y su especialidad
        "- un agente de investigación. Asigna tareas relacionadas con investigación a este agente\n"
        "- un agente de matemáticas. Asigna tareas relacionadas con matemáticas a este agente\n"
        
        # Regla importante: un agente a la vez (no en paralelo)
        "Asigna trabajo a un agente a la vez, no llames agentes en paralelo.\n"
        
        # El supervisor solo coordina, no hace el trabajo
        "No hagas ningún trabajo tú mismo."
    ),
    
    # Agrega mensajes de "handoff" (transferencia) de vuelta al supervisor
    # Esto permite que los agentes "reporten" al supervisor cuando terminen
    add_handoff_back_messages=True,
    
    # Modo de salida: devuelve el historial completo de mensajes
    # Alternativas: "final_answer" (solo la respuesta final)
    output_mode="full_history"
).compile()  # Compila el supervisor en un grafo ejecutable

# Muestra el supervisor compilado
supervisor

In [ ]:
# Invocación del sistema supervisor con una consulta compleja
# Esta consulta requiere AMBOS agentes: investigación Y matemáticas

response=supervisor.invoke({
    # Mensaje que combina dos tareas diferentes:
    # 1. Investigación: listar variantes de transformers (research_agent)
    # 2. Matemáticas: calcular 5 + 10 (math_agent)
    "messages": "lista todas las variantes de transformers en despliegues de producción del retriever y luego dime cuánto es 5 más 10"
})

# Flujo de ejecución esperado:
# 1. Supervisor analiza la consulta
# 2. Supervisor delega a research_agent (primera tarea)
# 3. research_agent usa internal_tool_1 para buscar transformers
# 4. research_agent reporta resultados al supervisor
# 5. Supervisor delega a math_agent (segunda tarea)
# 6. math_agent usa la función add(5, 10)
# 7. math_agent reporta resultado al supervisor
# 8. Supervisor consolida ambas respuestas

# La salida mostrará: "📚 Usando herramienta: InternalResearchNotes"

In [ ]:
# Muestra la respuesta completa del supervisor
# Incluye todo el historial de mensajes entre supervisor y agentes
# Útil para ver el flujo completo de delegación y respuestas
response

In [ ]:
# Extrae y muestra el contenido del último mensaje
# Este es el resultado final consolidado por el supervisor
# Debería contener:
# 1. Lista de variantes de transformers (de research_agent)
# 2. El resultado de 5 + 10 = 15 (de math_agent)
response["messages"][-1].content

### Equipos Jerárquicos de Agentes con RAG
En nuestro ejemplo anterior (Supervisor de Agentes), introdujimos el concepto de un solo nodo supervisor para enrutar trabajo entre diferentes nodos trabajadores.

Pero ¿qué pasa si el trabajo para un solo trabajador se vuelve demasiado complejo? ¿Qué pasa si el número de trabajadores se vuelve demasiado grande?

Para algunas aplicaciones, el sistema puede ser más efectivo si el trabajo se distribuye jerárquicamente.

Puedes hacer esto componiendo diferentes subgrafos y creando un supervisor de nivel superior, junto con supervisores de nivel medio.

In [ ]:
# Importaciones para el sistema jerárquico avanzado
from typing import Annotated, List

# WebBaseLoader: carga contenido desde URLs web
from langchain_community.document_loaders import WebBaseLoader

# TavilySearch: búsqueda web optimizada para IA
from langchain_tavily import TavilySearch

# tool decorator: convierte funciones Python en herramientas de LangChain
from langchain_core.tools import tool

# Creación de herramienta Tavily con límite de 5 resultados
tavily_tool = TavilySearch(max_results=5)

In [ ]:
# Definición de herramienta personalizada para scraping web
@tool
def scrape_webpages(urls: List[str]) -> str:
    """
    Usa requests y bs4 para scrapear las páginas web proporcionadas para obtener información detallada.
    
    Esta herramienta permite al agente acceder a contenido específico de URLs,
    complementando la búsqueda web general con información detallada de páginas específicas.
    
    Parámetros:
    - urls: lista de URLs a scrapear
    
    Retorna: contenido de todas las páginas concatenado y formateado
    """
    
    # Carga el contenido de las URLs usando WebBaseLoader
    # WebBaseLoader maneja automáticamente requests y BeautifulSoup
    loader = WebBaseLoader(urls)
    
    # .load() descarga y procesa todas las páginas
    docs = loader.load()
    
    # Formatea cada documento con su título y contenido
    # Usa tags XML-like para estructura clara
    return "\n\n".join(
        [
            # Para cada documento, crea un bloque estructurado
            f'<Document name="{doc.metadata.get("title", "")}">' +
            f'\n{doc.page_content}\n</Document>'
            for doc in docs
        ]
    )

In [ ]:
# Importaciones para herramientas de escritura de documentos
from pathlib import Path
from tempfile import TemporaryDirectory
from typing import Dict, Optional

# PythonREPL: permite ejecutar código Python dinámicamente
from langchain_experimental.utilities import PythonREPL

# TypedDict: para type hints de diccionarios con estructura específica
from typing_extensions import TypedDict

# Creación de directorio temporal para guardar documentos
# Esto mantiene los archivos aislados y se limpian automáticamente
_TEMP_DIRECTORY = TemporaryDirectory()
WORKING_DIRECTORY = Path(_TEMP_DIRECTORY.name)

# Herramienta 1: Crear esquemas/outlines
@tool
def create_outline(
    points: Annotated[List[str], "Lista de puntos principales o secciones."],
    file_name: Annotated[str, "Ruta del archivo para guardar el esquema."],
) -> Annotated[str, "Ruta del archivo de esquema guardado."]:
    """
    Crea y guarda un esquema/outline de documento.
    
    Útil para el agente note_taker que crea estructuras de documentos.
    """
    # Abre archivo en modo escritura
    with (WORKING_DIRECTORY / file_name).open("w") as file:
        # Enumera cada punto con formato "1. punto"
        for i, point in enumerate(points):
            file.write(f"{i + 1}. {point}\n")
    return f"Esquema guardado en {file_name}"

# Herramienta 2: Escribir documentos completos
@tool
def write_document(
    content: Annotated[str, "Contenido de texto a escribir en el documento."],
    file_name: Annotated[str, "Ruta del archivo para guardar el documento."],
) -> Annotated[str, "Ruta del archivo de documento guardado."]:
    """
    Crea y guarda un documento de texto.
    
    Útil para el agente doc_writer que escribe contenido completo.
    """
    # Escribe el contenido completo en el archivo
    with (WORKING_DIRECTORY / file_name).open("w") as file:
        file.write(content)
    return f"Documento guardado en {file_name}"

# Herramienta 3: Editar documentos existentes
@tool
def edit_document(
    file_name: Annotated[str, "Ruta del documento a editar."],
    inserts: Annotated[
        Dict[int, str],
        "Diccionario donde la clave es el número de línea (indexado desde 1) y el valor es el texto a insertar en esa línea.",
    ],
) -> Annotated[str, "Ruta del archivo de documento editado."]:
    """
    Edita un documento insertando texto en números de línea específicos.
    
    Útil para hacer correcciones o agregar contenido sin reescribir todo.
    """
    # Lee todas las líneas del archivo existente
    with (WORKING_DIRECTORY / file_name).open("r") as file:
        lines = file.readlines()

    # Ordena las inserciones por número de línea
    sorted_inserts = sorted(inserts.items())

    # Procesa cada inserción
    for line_number, text in sorted_inserts:
        # Verifica que el número de línea sea válido (1-indexado)
        if 1 <= line_number <= len(lines) + 1:
            # Inserta el texto en la posición especificada (convertido a 0-indexed)
            lines.insert(line_number - 1, text + "\n")
        else:
            # Retorna error si el número de línea está fuera de rango
            return f"Error: Número de línea {line_number} está fuera de rango."

    # Escribe las líneas modificadas de vuelta al archivo
    with (WORKING_DIRECTORY / file_name).open("w") as file:
        file.writelines(lines)

    return f"Documento editado y guardado en {file_name}"

In [ ]:
# Muestra el directorio de trabajo temporal donde se guardan los documentos
# Este es un directorio temporal que se limpia automáticamente
WORKING_DIRECTORY

In [ ]:
# Advertencia: Este código ejecuta código Python localmente,
# lo cual puede ser inseguro cuando no está en sandbox

# Creación de un intérprete Python REPL
# REPL = Read-Eval-Print Loop (Leer-Evaluar-Imprimir-Repetir)
repl = PythonREPL()


@tool
def python_repl_tool(
    code: Annotated[str, "El código Python a ejecutar para generar tu gráfico."],
):
    """
    Usa esto para ejecutar código Python. Si quieres ver la salida de un valor,
    debes imprimirlo con `print(...)`. Esto es visible para el usuario.
    
    Esta herramienta permite al agente chart_generator crear visualizaciones
    y realizar análisis de datos dinámicamente.
    
    Parámetros:
    - code: código Python a ejecutar (como string)
    
    Retorna: resultado de la ejecución o mensaje de error
    """
    try:
        # Intenta ejecutar el código usando el REPL
        result = repl.run(code)
    except BaseException as e:
        # Captura cualquier error durante la ejecución
        return f"Fallo al ejecutar. Error: {repr(e)}"
    
    # Retorna un mensaje formateado con el código y su salida
    return f"Ejecutado exitosamente:\n```python\n{code}\n```\nStdout: {result}"

In [ ]:
# Importaciones para el sistema jerárquico
from typing import List, Optional, Literal
from langchain_core.language_models.chat_models import BaseChatModel

from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.types import Command
from langchain_core.messages import HumanMessage, trim_messages


# Definición del estado personalizado que extiende MessagesState
class State(MessagesState):
    """
    Estado extendido que incluye campo 'next' para tracking.
    
    Campos:
    - messages: heredado de MessagesState (historial de mensajes)
    - next: indica el siguiente nodo a ejecutar
    """
    next: str  # Nombre del siguiente nodo/agente a ejecutar

In [ ]:
def make_supervisor_node(llm: BaseChatModel, members: list[str]) -> str:
    """
    Crea un nodo supervisor que enruta a trabajadores específicos.
    
    Esta función es el núcleo del patrón jerárquico. Crea un supervisor
    que puede delegar trabajo a múltiples agentes trabajadores.
    
    Parámetros:
    - llm: modelo de lenguaje para el supervisor
    - members: lista de nombres de agentes trabajadores disponibles
    
    Retorna: función del nodo supervisor configurada
    """
    
    # Opciones incluye "FINISH" más todos los trabajadores
    # "FINISH" indica que el trabajo está completo
    options = ["FINISH"] + members
    
    # Prompt del sistema para el supervisor
    system_prompt = (
        "Eres un supervisor encargado de gestionar una conversación entre los"
        f" siguientes trabajadores: {members}. Dada la siguiente solicitud del usuario,"
        " responde con el trabajador que debe actuar a continuación. Cada trabajador realizará una"
        " tarea y responderá con sus resultados y estado. Cuando termines,"
        " responde con FINISH."
    )

    # Definición de TypedDict para validación de respuesta estructurada
    class Router(TypedDict):
        """Trabajador al que enrutar a continuación. Si no se necesitan trabajadores, enrutar a FINISH."""
        next: Literal[*options]  # next debe ser uno de los valores en options

    def supervisor_node(state: State) -> Command[Literal[*members, "__end__"]]:
        """
        Un enrutador basado en LLM.
        
        Este nodo:
        1. Recibe el estado actual
        2. Usa el LLM para decidir el siguiente trabajador
        3. Retorna comando para navegar al siguiente nodo
        """
        # Construye lista de mensajes con prompt del sistema + historial
        messages = [
            {"role": "system", "content": system_prompt},
        ] + state["messages"]
        
        # Usa LLM con salida estructurada (Router TypedDict)
        # Esto fuerza al LLM a responder con formato: {"next": "nombre_trabajador"}
        response = llm.with_structured_output(Router).invoke(messages)
        
        # Extrae la decisión del siguiente nodo
        goto = response["next"]
        
        # Si el supervisor dice "FINISH", termina el grafo
        if goto == "FINISH":
            goto = END

        # Retorna comando con navegación al siguiente nodo
        return Command(goto=goto, update={"next": goto})

    # Retorna la función del nodo supervisor configurada
    return supervisor_node

In [ ]:
# Muestra la herramienta de recuperación interna
# Esta será usada por el equipo de investigación
internal_tool_1

In [ ]:
# Importaciones necesarias
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent


# === EQUIPO DE INVESTIGACIÓN ===

# Agente 1: search_agent (búsqueda general)
# Este agente busca información usando Tavily y documentos internos
search_agent = create_react_agent(llm, tools=[tavily_tool, internal_tool_1])
search_agent

def search_node(state: State) -> Command[Literal["supervisor"]]:
    """
    Nodo que ejecuta el agente de búsqueda.
    Siempre reporta de vuelta al supervisor del equipo de investigación.
    """
    # Invoca el agente de búsqueda con el estado actual
    result = search_agent.invoke(state)
    
    # Retorna comando con actualización y reporte al supervisor
    return Command(
        update={
            # Envuelve el mensaje en HumanMessage con nombre "search"
            "messages": [
                HumanMessage(content=result["messages"][-1].content, name="search")
            ]
        },
        # Queremos que nuestros trabajadores SIEMPRE "reporten" al supervisor cuando terminen
        goto="supervisor",
    )

### Subagente: web_scraper
# Agente 2: web_scraper_agent (scraping de páginas específicas)
# Este agente scrapea contenido detallado de URLs específicas
web_scraper_agent = create_react_agent(llm, tools=[scrape_webpages])


def web_scraper_node(state: State) -> Command[Literal["supervisor"]]:
    """
    Nodo que ejecuta el agente de scraping web.
    Siempre reporta de vuelta al supervisor del equipo de investigación.
    """
    # Invoca el agente de scraping
    result = web_scraper_agent.invoke(state)
    
    # Retorna comando con actualización y reporte al supervisor
    return Command(
        update={
            # Envuelve el mensaje con nombre "web_scraper"
            "messages": [
                HumanMessage(content=result["messages"][-1].content, name="web_scraper")
            ]
        },
        # Reporta al supervisor cuando termine
        goto="supervisor",
    )

In [ ]:
# Creación del supervisor del equipo de investigación
# Este supervisor coordina a search_agent y web_scraper_agent
research_supervisor_node = make_supervisor_node(llm, ["search", "web_scraper"])

# Muestra la función del supervisor de investigación
research_supervisor_node

In [ ]:
# === CONSTRUCCIÓN DEL GRAFO DEL EQUIPO DE INVESTIGACIÓN ===

# Creación del constructor del grafo para el equipo de investigación
research_builder = StateGraph(State)

# Agregado de nodos del equipo de investigación
# El supervisor coordina a dos trabajadores especializados
research_builder.add_node("supervisor", research_supervisor_node)  # Supervisor del equipo
research_builder.add_node("search", search_node)  # Trabajador de búsqueda
research_builder.add_node("web_scraper", web_scraper_node)  # Trabajador de scraping

# El flujo comienza con el supervisor del equipo de investigación
research_builder.add_edge(START, "supervisor")

# Compilación del subgrafo del equipo de investigación
research_graph = research_builder.compile()

# Muestra el grafo del equipo de investigación
research_graph

In [ ]:
# === HERRAMIENTA PARA LEER DOCUMENTOS ===

@tool
def read_document(
    file_name: Annotated[str, "Ruta del archivo para leer el documento."],
    start: Annotated[Optional[int], "La línea de inicio. Por defecto es 0"] = None,
    end: Annotated[Optional[int], "La línea final. Por defecto es None"] = None,
) -> str:
    """
    Lee el documento especificado.
    
    Permite leer todo el documento o solo un rango de líneas específico.
    Útil para que los agentes revisen documentos antes de editarlos.
    
    Parámetros:
    - file_name: nombre del archivo a leer
    - start: línea de inicio (opcional, por defecto 0)
    - end: línea final (opcional, por defecto None = hasta el final)
    
    Retorna: contenido del documento (completo o rango especificado)
    """
    # Abre y lee todas las líneas del archivo
    with (WORKING_DIRECTORY / file_name).open("r") as file:
        lines = file.readlines()
    
    # Si no se especifica start, comienza desde el principio
    if start is None:
        start = 0
    
    # Retorna las líneas desde start hasta end (o hasta el final si end es None)
    return "\n".join(lines[start:end])

In [ ]:
# === EQUIPO DE ESCRITURA DE DOCUMENTOS ===

# Agente 1: doc_writer_agent (escritor de documentos)
# Este agente puede leer, escribir y editar documentos completos
doc_writer_agent = create_react_agent(
    llm,
    tools=[write_document, edit_document, read_document],  # Herramientas de gestión de documentos
    prompt=(
        "Puedes leer, escribir y editar documentos basándote en los esquemas del tomador de notas. "
        "No hagas preguntas de seguimiento."
    ),
)


def doc_writing_node(state: State) -> Command[Literal["supervisor"]]:
    """
    Nodo que ejecuta el agente escritor de documentos.
    Reporta al supervisor del equipo de escritura cuando termine.
    """
    result = doc_writer_agent.invoke(state)
    return Command(
        update={
            "messages": [
                HumanMessage(content=result["messages"][-1].content, name="doc_writer")
            ]
        },
        goto="supervisor",
    )

# Agente 2: note_taking_agent (tomador de notas / creador de esquemas)
# Este agente crea outlines/esquemas que guían al escritor
note_taking_agent = create_react_agent(
    llm,
    tools=[create_outline, read_document],  # Herramientas de organización
    prompt=(
        "Puedes leer documentos y crear esquemas para el escritor de documentos. "
        "No hagas preguntas de seguimiento."
    ),
)


def note_taking_node(state: State) -> Command[Literal["supervisor"]]:
    """
    Nodo que ejecuta el agente tomador de notas.
    Reporta al supervisor del equipo de escritura cuando termine.
    """
    result = note_taking_agent.invoke(state)
    return Command(
        update={
            "messages": [
                HumanMessage(content=result["messages"][-1].content, name="note_taker")
            ]
        },
        goto="supervisor",
    )


# Agente 3: chart_generating_agent (generador de gráficos)
# Este agente crea visualizaciones y gráficos ejecutando código Python
chart_generating_agent = create_react_agent(
    llm, 
    tools=[read_document, python_repl_tool]  # Herramientas de análisis y visualización
)


def chart_generating_node(state: State) -> Command[Literal["supervisor"]]:
    """
    Nodo que ejecuta el agente generador de gráficos.
    Reporta al supervisor del equipo de escritura cuando termine.
    """
    result = chart_generating_agent.invoke(state)
    return Command(
        update={
            "messages": [
                HumanMessage(
                    content=result["messages"][-1].content, name="chart_generator"
                )
            ]
        },
        goto="supervisor",
    )

# Creación del supervisor del equipo de escritura
# Este supervisor coordina a doc_writer, note_taker y chart_generator
doc_writing_supervisor_node = make_supervisor_node(
    llm, ["doc_writer", "note_taker", "chart_generator"]
)

In [ ]:
# === CONSTRUCCIÓN DEL GRAFO DEL EQUIPO DE ESCRITURA ===

# Creación del constructor del grafo para el equipo de escritura de papers
paper_writing_builder = StateGraph(State)

# Agregado de nodos del equipo de escritura
# El supervisor coordina a tres trabajadores especializados
paper_writing_builder.add_node("supervisor", doc_writing_supervisor_node)  # Supervisor del equipo
paper_writing_builder.add_node("doc_writer", doc_writing_node)  # Escritor de documentos
paper_writing_builder.add_node("note_taker", note_taking_node)  # Tomador de notas
paper_writing_builder.add_node("chart_generator", chart_generating_node)  # Generador de gráficos

# El flujo comienza con el supervisor del equipo de escritura
paper_writing_builder.add_edge(START, "supervisor")

# Compilación del subgrafo del equipo de escritura
paper_writing_graph = paper_writing_builder.compile()

# Muestra el grafo del equipo de escritura
paper_writing_graph

In [ ]:
# === SUPERVISOR DE NIVEL SUPERIOR ===

# Importación de BaseMessage
from langchain_core.messages import BaseMessage

# Creación del supervisor de equipos (nivel superior)
# Este supervisor coordina a los dos equipos completos:
# - research_team (búsqueda + scraping)
# - writing_team (notas + escritura + gráficos)
teams_supervisor_node = make_supervisor_node(llm, ["research_team", "writing_team"])

# Muestra la función del supervisor de equipos
teams_supervisor_node

In [ ]:
# === FUNCIONES DE LLAMADA A EQUIPOS ===

def call_research_team(state: State) -> Command[Literal["supervisor"]]:
    """
    Invoca el subgrafo completo del equipo de investigación.
    
    Este nodo actúa como "proxy" que:
    1. Toma el último mensaje del estado
    2. Lo pasa al equipo de investigación completo
    3. Espera a que el equipo termine su trabajo
    4. Retorna los resultados al supervisor de equipos
    """
    # Invoca el grafo completo del equipo de investigación
    # Solo pasa el último mensaje para evitar duplicación
    response = research_graph.invoke({"messages": state["messages"][-1]})
    
    # Retorna los resultados al supervisor de equipos
    return Command(
        update={
            # Marca el mensaje como proveniente del "research_team"
            "messages": [
                HumanMessage(
                    content=response["messages"][-1].content, name="research_team"
                )
            ]
        },
        goto="supervisor",  # Siempre reporta al supervisor de equipos
    )


def call_paper_writing_team(state: State) -> Command[Literal["supervisor"]]:
    """
    Invoca el subgrafo completo del equipo de escritura.
    
    Similar a call_research_team, pero para el equipo de escritura.
    """
    # Invoca el grafo completo del equipo de escritura
    response = paper_writing_graph.invoke({"messages": state["messages"][-1]})
    
    # Retorna los resultados al supervisor de equipos
    return Command(
        update={
            # Marca el mensaje como proveniente del "writing_team"
            "messages": [
                HumanMessage(
                    content=response["messages"][-1].content, name="writing_team"
                )
            ]
        },
        goto="supervisor",  # Siempre reporta al supervisor de equipos
    )


# === CONSTRUCCIÓN DEL GRAFO SUPERIOR JERÁRQUICO ===

# Creación del constructor del grafo de nivel superior
super_builder = StateGraph(State)

# Agregado de nodos del nivel superior
# Este grafo coordina a los dos equipos completos
super_builder.add_node("supervisor", teams_supervisor_node)  # Supervisor de equipos
super_builder.add_node("research_team", call_research_team)  # Nodo que invoca equipo de investigación
super_builder.add_node("writing_team", call_paper_writing_team)  # Nodo que invoca equipo de escritura

# El flujo comienza con el supervisor de equipos
super_builder.add_edge(START, "supervisor")

# Compilación del grafo jerárquico completo
super_graph = super_builder.compile()

# Muestra el grafo jerárquico completo
super_graph

In [ ]:
# === INVOCACIÓN DEL SISTEMA JERÁRQUICO COMPLETO ===

# Invoca el sistema jerárquico multi-agente completo con una tarea compleja
response=super_graph.invoke(
    {
        # La consulta solicita escribir sobre variantes de transformers
        # Esto requiere:
        # 1. Equipo de investigación: buscar información (search + web_scraper)
        # 2. Equipo de escritura: crear outline (note_taker) → escribir documento (doc_writer)
        "messages": [
            ("user", "Escribe sobre variantes de transformers en despliegues de producción.")
        ],
    }
)

# Flujo de ejecución esperado:
# 1. Supervisor de equipos analiza la tarea
# 2. Delega a research_team
#    2.1. Supervisor de investigación coordina a search y web_scraper
#    2.2. Recopilan información sobre transformers
#    2.3. Reportan resultados al supervisor de equipos
# 3. Supervisor de equipos delega a writing_team
#    3.1. Supervisor de escritura coordina a note_taker, doc_writer y chart_generator
#    3.2. note_taker crea outline del documento
#    3.3. doc_writer escribe el documento completo
#    3.4. Reportan resultados al supervisor de equipos
# 4. Supervisor de equipos confirma finalización

In [ ]:
# Muestra la respuesta completa del sistema jerárquico
# Incluye todo el historial de mensajes entre:
# - Supervisor de equipos
# - Equipo de investigación (y sus subagentes)
# - Equipo de escritura (y sus subagentes)
response

In [ ]:
# Extrae y muestra el contenido del último mensaje
# Este es el mensaje final del equipo de escritura confirmando que:
# 1. El documento ha sido creado
# 2. El outline ha sido guardado
# 3. El trabajo está completo
response["messages"][-1].content